<a href="https://colab.research.google.com/github/AdarshKhatri01/DBMS-Notes/blob/main/Transactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transaction in DBMS**  
A **transaction** is a sequence of one or more database operations (like `INSERT`, `UPDATE`, `DELETE`, `SELECT`) executed as a single unit of work. Transactions ensure that the database remains **consistent and reliable**, even in the event of system failures.  

#### **Example of a Transaction:**  
Suppose you are transferring ₹1000 from Account A to Account B:  
1. **Withdraw ₹1000 from Account A**  
   ```sql
   UPDATE accounts SET balance = balance - 1000 WHERE account_id = 'A';
   ```
2. **Deposit ₹1000 into Account B**  
   ```sql
   UPDATE accounts SET balance = balance + 1000 WHERE account_id = 'B';
   ```
3. **Commit the transaction** (if both steps are successful)  
   ```sql
   COMMIT;
   ```
4. **Rollback if an error occurs**  
   ```sql
   ROLLBACK;
   ```  
If one step fails (e.g., system crash after withdrawal but before deposit), the transaction is **rolled back** to prevent data inconsistency.

---

# **ACID Properties of Transactions**  
To maintain **database integrity and reliability**, transactions follow the **ACID properties**:  

1. **Atomicity (A)** – *All or nothing!*  
   - Either **all** operations in a transaction execute successfully, or **none** of them are applied.  
   - Example: If money is deducted from Account A but not credited to Account B due to a failure, the whole transaction is rolled back.

2. **Consistency (C)** – *Database remains valid before and after the transaction.*  
   - A transaction ensures that the database remains in a valid state, following all integrity constraints.  
   - Example: If a transaction deducts ₹1000 from A, it must **add ₹1000 to B**, ensuring no data is lost.

3. **Isolation (I)** – *Transactions do not interfere with each other.*  
   - If multiple transactions run at the same time, they should not affect each other’s execution.  
   - Example: If two users transfer money at the same time, one transaction should **not see** half-completed changes of the other.

4. **Durability (D)** – *Once committed, the changes are permanent.*  
   - Even if a system crash occurs after a transaction is committed, the changes **must not be lost**.  
   - Example: If ₹1000 is transferred successfully and the system crashes, the transfer should **not be reversed**.

---

### **Conclusion**  
- Transactions ensure **data consistency and integrity** using **ACID properties**.  

# **Transaction States in DBMS**  

## **1. What is a Transaction?**  
A **transaction** is a **sequence of database operations** that must be executed as a **single unit**. It ensures **Atomicity, Consistency, Isolation, and Durability (ACID properties).**  

A transaction can be in different **states** during its execution, from start to commit or rollback.

---

## **2. Transaction States with Diagram**
A transaction in DBMS **moves through different states** before being committed or aborted.

### **Transaction States Diagram**
```plaintext
        +-----------+       +------------+
        |   Active  | ----> | Partially  |  
        |           |       | Committed  |  
        +-----------+       +------------+
              |                     |
              |                     v
              |              +--------------+
              |              |   Committed  |
              |              +--------------+
              v
       +-----------+       +------------+
       |   Failed  | ----> |   Aborted   |
       +-----------+       +------------+
```

---

### **3. Explanation of Transaction States**  

### **1️⃣ Active State**  
- The transaction **starts** and is executing operations (INSERT, UPDATE, DELETE, etc.).
- If any failure occurs, the transaction moves to the **Failed state**.

✅ **Example:**  
```sql
START TRANSACTION;
UPDATE Account SET Balance = Balance - 500 WHERE Account_ID = 101;
```
🔹 The transaction is still active as it hasn't been committed.

---

### **2️⃣ Partially Committed State**  
- The transaction has **executed all operations**, but changes are not yet **permanent**.  
- It enters this state **just before commit**.

✅ **Example:**  
```sql
UPDATE Account SET Balance = Balance + 500 WHERE Account_ID = 102;
```
🔹 The second update is done, but the transaction is **not yet committed**.

---

### **3️⃣ Committed State**  
- The transaction **is successfully completed**.  
- Changes are **permanently saved in the database**.  
- The system ensures **Durability (ACID property)**.

✅ **Example:**  
```sql
COMMIT;
```
🔹 The transaction moves to the **Committed state** after commit.

---

### **4️⃣ Failed State**  
- If the transaction **encounters an error** (e.g., system crash, constraint violation), it enters the **Failed state**.  
- Changes made by the transaction are **invalid** and should not be saved.

❌ **Example:**  
- A system crash occurs before commit.
- A constraint violation happens (e.g., inserting duplicate PRIMARY KEY).

---

### **5️⃣ Aborted State**  
- If a transaction **fails**, it must be **rolled back** (undo changes).  
- The transaction is either restarted or permanently terminated.

✅ **Example:**  
```sql
ROLLBACK;
```
🔹 All changes made by the transaction are **undone**, and the database remains **unchanged**.

---

## **4. Summary of Transaction States**
| **State** | **Description** |
|-----------|----------------|
| **Active** | Transaction is in progress. |
| **Partially Committed** | All operations are executed, but changes are not saved yet. |
| **Committed** | Changes are permanently saved. |
| **Failed** | An error or system failure occurs. |
| **Aborted** | Changes are rolled back, and the transaction is terminated. |

---

## **5. Conclusion**
Transactions ensure **data consistency and reliability** in DBMS. The **state diagram** helps understand how a transaction **progresses from execution to commit or rollback**.

# **Concurrency and Its Problems in DBMS**

## **1. What is Concurrency in DBMS?**  
**Concurrency** in DBMS refers to the **simultaneous execution of multiple transactions** to improve **performance and resource utilization**. It ensures multiple users can access the database **at the same time** without conflicts.

✅ **Advantages of Concurrency:**  
- **Faster processing** (Multiple users can work simultaneously).  
- **Efficient resource utilization** (CPU and I/O usage is maximized).  
- **Improved system throughput** (More transactions processed per second).  

❌ **Challenges:** Concurrency can lead to problems like **lost updates, dirty reads, inconsistent retrievals, and uncommitted dependencies**.

---

## **2. Concurrency Problems in DBMS**
When multiple transactions run **concurrently**, they may interfere with each other and cause **data inconsistency**. The main problems are:

### **1️⃣ Lost Update Problem**  
- Occurs when **two transactions update the same data item simultaneously**, and one update **overwrites the other**.  

✅ **Example (T1 and T2 running concurrently):**  
```plaintext
T1: Read(A) → A = A + 500 → Write(A)
T2: Read(A) → A = A - 200 → Write(A)
```
**Initial Value of A = 1000**  
Execution Order:
```plaintext
T1 reads A = 1000
T2 reads A = 1000
T1 updates A = 1000 + 500 = 1500
T2 updates A = 1000 - 200 = 800  (Overwrites T1’s update!)
```
🔴 **Final Value of A = 800 (Wrong!)**  
✅ **Expected Correct Value = 1300**  

### **Solution:** Use **Locking Mechanisms** (Two-Phase Locking) to prevent updates before previous transactions finish.

---

### **2️⃣ Dirty Read Problem**  
- Occurs when a transaction **reads uncommitted data** from another transaction that later **gets rolled back**.  

✅ **Example:**  
```plaintext
T1: Update A = 1000 → 1500 (Uncommitted)
T2: Read(A) → A = 1500
T1: Rollback (A should be 1000, but T2 already read 1500!)
```
🔴 **Problem:** T2 read incorrect data that should not have existed.  

### **Solution:** Use **Strict Isolation Levels** (e.g., **READ COMMITTED** isolation).

---

### **3️⃣ Uncommitted Dependency (Dirty Write)**  
- Happens when a **transaction writes to a data item that is later rolled back**.  

✅ **Example:**  
```plaintext
T1: Update A = 1000 → 1500 (Uncommitted)
T2: Update A = 1500 → 2000 (Using T1’s uncommitted value!)
T1: Rollback (A should be 1000, but T2 already used 1500!)
```
🔴 **Problem:** T2 used an invalid value that got rolled back.  

### **Solution:** Use **STRICT SCHEDULING** (Do not allow updates until commit).

---

### **4️⃣ Inconsistent Retrieval (Non-Repeatable Read)**
- Occurs when a transaction reads a **data item multiple times** and gets **different values** due to another transaction updating it.  

✅ **Example:**  
```plaintext
T1: Read(A) → A = 1000
T2: Update A = 1000 → 1500 → Commit
T1: Read(A) again → A = 1500 (Value changed!)
```
🔴 **Problem:** T1 gets different values for A during execution.  

### **Solution:** Use **Repeatable Read Isolation Level** (Locks data during read).

---

### **5️⃣ Phantom Read Problem**
- Occurs when a transaction **retrieves different sets of rows** in repeated queries due to another transaction **inserting/deleting** rows.  

✅ **Example:**  
```plaintext
T1: SELECT COUNT(*) FROM Students → Returns 100 records
T2: INSERT INTO Students VALUES (New Student)
T1: SELECT COUNT(*) FROM Students → Returns 101 records
```
🔴 **Problem:** T1 sees different results for the same query.  

### **Solution:** Use **Serializable Isolation** (Locks the entire table to prevent inserts).

---

## **3. Summary of Concurrency Problems**
| **Problem**            | **Description** | **Example Scenario** | **Solution** |
|------------------------|---------------|----------------|------------|
| **Lost Update** | One transaction overwrites another's changes. | Two users update the same bank balance at the same time. | **Two-Phase Locking (2PL)** |
| **Dirty Read** | A transaction reads uncommitted data from another. | Reading an **uncommitted salary update** before rollback. | **Read Committed Isolation** |
| **Uncommitted Dependency** | A transaction updates using an uncommitted value. | Updating a product price based on a failed transaction. | **Strict Scheduling** |
| **Inconsistent Retrieval** | Data changes during a transaction’s execution. | A report shows different total values each time it is read. | **Repeatable Read Isolation** |
| **Phantom Read** | New rows appear/disappear between queries. | Running an attendance count before/after an insert. | **Serializable Isolation** |

---

## **4. Conclusion**
- Concurrency **improves performance** but must be handled carefully to prevent **data inconsistency**. The best approach is to use **proper locking mechanisms and isolation levels** based on the application’s requirements.
- Concurrency control ensures data consistency and correctness when multiple transactions execute simultaneously.